Imports

In [1]:
import numpy as np
from utils.dawnplot import *
from utils.dawnfrb import FRBInfo

catalog = FRBInfo(
    '/home/jovyan/work/chime_data/21.0007/',
    '/home/jovyan/work/chime_data/23.0004/',
    remove_duplicates=True
)

In [2]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/conda/envs/umap/lib
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/conda/envs/umap/lib
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Get Data

In [3]:
timeSeries = catalog['ts']
modelTimeSeries = catalog['model_ts']

pad_len = max([len(x) for x in timeSeries])
timeSeries = [np.concatenate([x, np.zeros(pad_len - len(x))]) for x in timeSeries]
timeSeriesNormed = [series / np.max(series) for series in timeSeries]

In [9]:
from umap.parametric_umap import ParametricUMAP

shape = (len(timeSeries[0]),1)
n_components = 2
encoder = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=shape),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(n_components, activation='linear')
], name='encoder')

plot = BokehInterface(
    views=[
        # EmbeddingView(
        #     title='raw_ts_para',
        #     data=timeSeries,
        #     reducer=ParametricUMAP(
        #         n_training_epochs=2,
        #         verbose=True,
        #     ),
        # ),
        EmbeddingView(
            title='raw_ts_para_cnn',
            data=timeSeries,
            reducer=ParametricUMAP(
                encoder=encoder,
                dims=shape,
                verbose=True,
            ),
        ),
        EmbeddingView(
            title='raw_ts_normed_para_cnn',
            data=timeSeriesNormed,
            reducer=ParametricUMAP(
                encoder=encoder,
                dims=shape,
                verbose=True,
            ),
        ),
    ],
    plot_elements=[
        ColorTooltipData(
            title='width_ms',
            data=[i*1000 for i in catalog['width_fitb']],
        ),
        ColorTooltipData(
            title='dm',
            data=catalog['dm_fitb'],
        ),
        ColorTooltipData(
            title='n_subbursts',
            data=catalog['n_subbursts'],
        ),
        ColorMap(
            title='scattering_ms',
            data=[i*1000 for i in catalog['scat_time']],
        ),
        ColorMap(
            title='argmax',
            data=[np.argmax(i) for i in timeSeries],
        ),
        ColorMap(
            title='scatterfit',
            data=["✔️" if i else "❌" for i in catalog['scatterfit']],
            high_transparent=False,
        ),
        ColorMap(
            title='known_repeater',
            data=["✔️" if rp != -9999.0 else "❌" for rp in catalog['repeater_name']],
            high_transparent=False,
        ),
        ColorMap(
            title='ts_embedding_label',
            data=[str(l) for l in catalog['ts_embedding_label']],
        ),
        TooltipData(
            title='tns_name',
            data=catalog['tns_name'],
        ),
        TooltipGraph(
            timeSeries,
            title='raw_ts',
        ),
    ],
)

ParametricUMAP(dims=(762, 1), encoder=<keras.engine.sequential.Sequential object at 0x7f3c224049a0>, optimizer=<keras.optimizer_v2.adam.Adam object at 0x7f40200b4520>)
Tue Aug 22 16:35:04 2023 Construct fuzzy simplicial set
Tue Aug 22 16:35:04 2023 Finding Nearest Neighbors
Tue Aug 22 16:35:04 2023 Finished Nearest Neighbor Search
Tue Aug 22 16:35:04 2023 Construct embedding
Epoch 1/10
335/335 [==============================] - 22s 65ms/step - loss: 0.2579
Epoch 2/10
335/335 [==============================] - 22s 65ms/step - loss: 0.2312
Epoch 3/10
335/335 [==============================] - 22s 64ms/step - loss: 0.2283
Epoch 4/10
335/335 [==============================] - 22s 65ms/step - loss: 0.2256
Epoch 5/10
335/335 [==============================] - 22s 65ms/step - loss: 0.2234
Epoch 6/10
335/335 [==============================] - 22s 65ms/step - loss: 0.2201
Epoch 7/10
335/335 [==============================] - 22s 64ms/step - loss: 0.2263
Epoch 8/10
335/335 [=====================

In [10]:
plot.PLOT_DF['marker'] = ['star' if rp != -9999.0 else 'circle' for rp in catalog['repeater_name']]
plot.PLOT_DF['size'] = [20 if rp != -9999.0 else 6 for rp in catalog['repeater_name']]
plot.init_plot_elements()
plot.plotPoints.glyph.size='size'
plot.plotPoints.glyph.marker = 'marker'
# plot.plotPoints.glyph.line_alpha=1
plot.plotPoints.glyph.line_color='black'
# plot.plotPoints.glyph.fill_alpha=1
# plot.plotPoints.glyph.line_width='outline'
plot.show()

Loading BokehJS ...

In [ ]:
import bokeh
bokeh.output_file(title="Bokeh Plot", filename='bokeh_saves/test.html')